In [1]:
from riotwatcher import RiotWatcher,LolWatcher, ApiError
import pandas as pd
import numpy as np

In [2]:
#para medir la fuerza
count_fuerza = 120
#para trackear partidas
count = 10
#nombre invocador
user_name = 'the ghold liom  '
#region
region = 'la2'
#cola de traqueos ranked/flex/normal/aram
cola_raw = 'ranked'
# Usa tu clave de API aquí 
#ESTA API CAMBIA DIARIAMENTE, SE DEBE ACTUALIZAR
api_key = 'RGAPI-5dbbd65c-cc2c-4eae-8073-df2dac2249bb'


In [3]:
# Crea un objeto de LolWatcher
lol_watcher = LolWatcher(api_key)

NORMAL_RECL_ = 400;
RANKED_SOLO_ = 420;
NORMAL_BLIND_ = 430;
RANKED_FLEX_ = 440;
ARAM_ = 450;
dicc_cola = {'normal':NORMAL_RECL_,
'ranked':RANKED_SOLO_ ,
'blind':NORMAL_BLIND_ ,
'flex':RANKED_FLEX_,
'aram':ARAM_ }
cola = dicc_cola[cola_raw]

#usuario = lol_watcher.summoner.by_name(region, user_name)

In [4]:
# Crea un objeto de LolWatcher
lol_watcher = LolWatcher(api_key)
riot_watcher = RiotWatcher(api_key)

NORMAL_RECL_ = 400;
RANKED_SOLO_ = 420;
NORMAL_BLIND_ = 430;
RANKED_FLEX_ = 440;
ARAM_ = 450;
dicc_cola = {'normal':NORMAL_RECL_,
'ranked':RANKED_SOLO_ ,
'blind':NORMAL_BLIND_ ,
'flex':RANKED_FLEX_,
'aram':ARAM_ }
cola = dicc_cola[cola_raw]

PUUID_USER = riot_watcher.account.by_riot_id(region='AMERICAS',game_name=user_name,tag_line='LAS')['puuid']

usuario = lol_watcher.summoner.by_puuid(region, PUUID_USER)

In [5]:
def get_last_ranked(usuario,region=region,queue=RANKED_SOLO_):
    return lol_watcher.match.matchlist_by_puuid(region, usuario["puuid"],count=count,queue=queue)[0]
def get_last_match(usuario,region=region):
    return lol_watcher.match.matchlist_by_puuid(region, usuario["puuid"],count=count)[0]
def get_previous_matches(summoner_puuid, region, match_id,queue=None,count=10):
    try:
        #ARREGLAR ESTA LINEA PARA QUE BUSQUE EL MATCHID
        previous_matches = []
        found_match = False
        idx_init=0
        search = min([count+20,100])
        keep = True
        while (not found_match) or keep:
            matchlist = lol_watcher.match.matchlist_by_puuid(region, summoner_puuid,count = search,start=idx_init,queue=queue)
            if len(matchlist) == 0:
                break
            elif keep:
                for match in matchlist:
                    if match == match_id:
                        found_match = True
                    elif found_match:
                        previous_matches.append(match)
                        if len(previous_matches) >= count:
                            keep = False
                            break
                idx_init += search
            
        return previous_matches
    except ApiError as err:
        print(f"Error retrieving previous matches: {err}")
        return None
def get_match_stats(matchid,region,include_teams = False):
    if isinstance(matchid,str):
        matchid = [matchid]
    elif isinstance(matchid,list):
        pass
    else:
        raise TypeError("Solo formato string o list son permitidos.")
    matches_dict = dict()
    i = 0
    for m_id in matchid:
        if len(matchid) > 100 and (i+1)%50 == 0:
            print(f"Obteniedo estadisticas para {i+1} partidas\n")
        elif len(matchid) <= 100 and i%10 == 0:
            print(f"Obteniedo estadisticas para {i+1} partidas\n")
        #print información
        #print(f"Obteniedo estadisticas de la partida {m_id}\n")
        matches_dict[m_id] = lol_watcher.match.by_id(region, m_id)['info']['participants']
        if include_teams:
            print("incluyendo información de teams\n")
            matches_dict[m_id].append(lol_watcher.match.by_id(region, m_id)['info']['teams'])
        i += 1
    return matches_dict

        

In [6]:
match_seed = get_last_ranked(usuario,region=region)
last_rankeds = get_previous_matches(usuario['puuid'], region=region,match_id=match_seed,queue=RANKED_SOLO_,count=count_fuerza)

L = [match_seed] + get_previous_matches(usuario['puuid'],region,match_seed,count=count_fuerza)
match_stats = get_match_stats(L,region)



Obteniedo estadisticas para 50 partidas

Obteniedo estadisticas para 100 partidas



In [7]:
class force(object):
    def get_match_stats(self,matchid,region,include_teams = False):
        if isinstance(matchid,str):
            matchid = [matchid]
        elif isinstance(matchid,list):
            pass
        else:
            raise TypeError("Solo formato string o list son permitidos.")
        matches_dict = dict()
        i = 0
        for m_id in matchid:
            if len(matchid) > 100 and (i+1)%50 == 0:
                print(f"Obteniedo estadisticas para {i+1} partidas\n")
            elif len(matchid) <= 100 and i%10 == 0:
                print(f"Obteniedo estadisticas para {i+1} partidas\n")
            #print información
            #print(f"Obteniedo estadisticas de la partida {m_id}\n")
            matches_dict[m_id] = lol_watcher.match.by_id(region, m_id)['info']['participants']
            if include_teams:
                print("incluyendo información de teams\n")
                matches_dict[m_id].append(lol_watcher.match.by_id(region, m_id)['info']['teams'])
            i += 1
        return matches_dict
    
    def get_net_force(self,user_name,lista_match,region,include_teams = False,show = False):
        print(f"Obteniendo estadisticas de las partidas, numero de partidas = {len(lista_match)}\n")
        match_stats = get_match_stats(lista_match,region,include_teams=include_teams)
        print('Estadisticas obtenidas\n')

        print('Procesando información\n')
        LL = list()
        for m_id,dicc in match_stats.items():
            df_list = [pd.DataFrame({x:[y] for x,y in dicc[n].items() if x not in ['challenges','perks']}) for n in range(10)]
            df = pd.concat(df_list)
            df['matchid'] = m_id
            df = df.reset_index().drop('index',axis=1)
            LL.append(df)
        df_final = pd.concat(LL).reset_index().drop('index',axis=1)
        cols = ['matchid'] + df_final.columns.tolist()[:-1]
        df_final = df_final[cols]
        
        print('Borrando duplicados..\n')
        
        # Paso 1: Encuentra las columnas duplicadas
        columnas_duplicadas = df_final.columns[df_final.columns.duplicated()]
        todas_identicas = all([(df_final[x].iloc[:, 0] == df_final[x].iloc[:, 1]).all() for x in columnas_duplicadas])
        if todas_identicas:
            print('Todas las columnas duplicadas son identicas, se proceden a borrar\n')
            df_final = df_final.loc[:, ~df_final.columns.duplicated()]
        else:
            print('Existen columnas duplicadas no identicas, chequear las siguientes columnas:\n')
            for columna in columnas_duplicadas:
                son_identicas = df_final[columna].iloc[:, 0] == df_final[columna].iloc[:, 1]
                if not son_identicas.all():
                    print(f"{columna}")
        

        print('Calculando mascaras\n')
        print(df_final.columns.tolist())        
        mask1 = df_final['riotIdGameName'] == user_name
        mask2 = df_final['win'] == True
        mask3 = df_final['win'] == False
        mask4 = df_final['matchid'].isin( df_final.loc[mask1&mask3]['matchid'].unique().tolist()) == True
        mask5 = df_final['matchid'].isin( df_final.loc[mask1&mask2]['matchid'].unique().tolist()) == True

        print('Mascaras calculadas\n')
        
        print('Calculando fuerzas\n')


        positive_force = df_final.loc[mask1&mask2][['championName']].value_counts()/df_final.loc[mask1][['matchid']].size
        negative_force = (df_final.loc[~mask1&mask4]
         .loc[lambda x: x['win'] == True][['championName']]
         .value_counts()/df_final.loc[mask1][['matchid']].size)
        helpful_force = (df_final.loc[~mask1&mask5]
         .loc[lambda x: x['win'] == True][['championName']]
         .value_counts()/df_final.loc[mask1][['matchid']].size)
        useless_force = (df_final.loc[~mask1&mask4]
         .loc[lambda x: x['win'] == False][['championName']]
         .value_counts()/df_final.loc[mask1][['matchid']].size)
        

        print('Fuerzas calculadas\n')
        

        print('Calculando fuerza neta\n')

        positive_force.sort_values(ascending=False,inplace=True)
        negative_force.sort_values(ascending=False,inplace=True)

        idx_resta = [(x[0],y[0]) for x in positive_force.index.tolist() for y in negative_force.index.tolist()]
        resta = list()
        for x in positive_force:
            for y in negative_force:
                resta.append(x-y)
        net_force = pd.DataFrame(resta,index = idx_resta,columns = ['net_force'])
        net_force['pick'] = [tupla[0] for tupla in idx_resta]
        net_force['counter'] = [tupla[1] for tupla in idx_resta]

        print('Fueza neta calculada con éxito\n')

        if show:
            print(net_force.rename(columns = {'net_force':'pick_vs_counter'}))

        return {'net_force':net_force,
                'positive_force':positive_force,
                'negative_force':negative_force,
                'helpful_force':helpful_force,
                'useless_force':useless_force}
    
    def __init__(self,user_name,lista_matches,region):
        self.lista_matches = lista_matches
        self.region = region
        self.force_dict = self.get_net_force(user_name,self.lista_matches,region)
        self.positive = self.force_dict['positive_force']
        self.negative = self.force_dict['negative_force']
        self.helpful = self.force_dict['helpful_force']
        self.useless = self.force_dict['useless_force']
        self.netforce = self.force_dict['net_force']
        self.strongest_pick = self.positive.to_frame().reset_index().iloc[0]['championName']
        self.strongest_counter = self.negative.to_frame().reset_index().iloc[0]['championName']
    
    def utility(self):
        df1 = fuerzas.helpful.to_frame().reset_index().rename(columns = {0:'force'})
        df2 = fuerzas.useless.to_frame().reset_index().rename(columns = {0:'force'})
        df3 = df1.merge(df2,on = 'championName',how='outer',suffixes = ('_h','_u')).fillna(0)
        df3['utility'] = df3['force_h'] - df3['force_u']
        return df3.sort_values('utility',ascending = False)





In [8]:
%%time
fuerzas = force(user_name,last_rankeds,region=region)

Obteniendo estadisticas de las partidas, numero de partidas = 120

Obteniedo estadisticas para 50 partidas

Obteniedo estadisticas para 100 partidas

Estadisticas obtenidas

Procesando información

Borrando duplicados..

Todas las columnas duplicadas son identicas, se proceden a borrar

Calculando mascaras

['matchid', 'allInPings', 'assistMePings', 'assists', 'baronKills', 'basicPings', 'bountyLevel', 'champExperience', 'champLevel', 'championId', 'championName', 'championTransform', 'commandPings', 'consumablesPurchased', 'damageDealtToBuildings', 'damageDealtToObjectives', 'damageDealtToTurrets', 'damageSelfMitigated', 'dangerPings', 'deaths', 'detectorWardsPlaced', 'doubleKills', 'dragonKills', 'eligibleForProgression', 'enemyMissingPings', 'enemyVisionPings', 'firstBloodAssist', 'firstBloodKill', 'firstTowerAssist', 'firstTowerKill', 'gameEndedInEarlySurrender', 'gameEndedInSurrender', 'getBackPings', 'goldEarned', 'goldSpent', 'holdPings', 'individualPosition', 'inhibitorKills', 

In [9]:
fuerzas.positive.to_frame().rename(columns={0:'positive force'})[:5]

,positive force
championName,
Sett,0.033333
Jax,0.025000
Pantheon,0.025000
Kayn,0.025000
Volibear,0.025000


In [10]:
fuerzas.negative.to_frame().rename(columns={0:'negative force'})[:5]

,negative force
championName,
Ezreal,0.075000
Lux,0.075000
MissFortune,0.066667
Pyke,0.066667
Yone,0.058333


In [11]:
#TU PICK
#default strongest_pick
your_pick = fuerzas.strongest_pick

In [12]:
#EJECUTAME SI QUIERES SABER LA LISTA DE TUS PICKS
fuerzas.netforce['pick'].sort_values().unique()

array(['Aatrox', 'Ashe', 'Camille', 'Darius', 'Diana', 'DrMundo', 'Ekko',
       'Gangplank', 'Gwen', 'Jax', 'Jhin', 'Jinx', 'Katarina', 'Kayle',
       'Kayn', 'Khazix', 'Malphite', 'Malzahar', 'MissFortune',
       'Mordekaiser', 'Nasus', 'Olaf', 'Pantheon', 'Renekton', 'Samira',
       'Senna', 'Sett', 'Skarner', 'Swain', 'Sylas', 'Taric', 'Udyr',
       'Vayne', 'Vex', 'Volibear', 'Warwick', 'Yasuo', 'Yone', 'Zed'],
      dtype=object)

In [13]:
if your_pick in['',None]:
    aa = fuerzas.netforce.loc[lambda x: x['pick']==fuerzas.strongest_pick].sort_values('net_force')[:5]
else:
    aa = fuerzas.netforce.loc[lambda x: x['pick']==your_pick].sort_values('net_force')[:5]
aa

,net_force,pick,counter
"(Sett, Ezreal)",-0.041667,Sett,Ezreal
"(Sett, Lux)",-0.041667,Sett,Lux
"(Sett, MissFortune)",-0.033333,Sett,MissFortune
"(Sett, Pyke)",-0.033333,Sett,Pyke
"(Sett, Yone)",-0.025000,Sett,Yone


In [14]:
#TU COUNTER
#dafult strongest_counter
your_counter = fuerzas.strongest_counter

In [15]:
#EJECUTAME SI QUIERES SABER LA LISTA DE COUNTER
fuerzas.netforce['counter'].sort_values().unique()

array(['Aatrox', 'Ahri', 'Akali', 'Akshan', 'Aphelios', 'Ashe',
       'AurelionSol', 'Azir', 'Belveth', 'Blitzcrank', 'Brand', 'Briar',
       'Caitlyn', 'Chogath', 'Darius', 'Diana', 'DrMundo', 'Ekko',
       'Elise', 'Evelynn', 'Ezreal', 'FiddleSticks', 'Fiora', 'Fizz',
       'Gangplank', 'Garen', 'Gnar', 'Gragas', 'Graves', 'Heimerdinger',
       'Hwei', 'Illaoi', 'Irelia', 'Ivern', 'Janna', 'Jax', 'Jayce',
       'Jhin', 'Jinx', 'KSante', 'Kaisa', 'Karma', 'Karthus', 'Kassadin',
       'Katarina', 'Kayle', 'Kayn', 'Khazix', 'Kindred', 'Kled',
       'Leblanc', 'LeeSin', 'Leona', 'Lillia', 'Lucian', 'Lulu', 'Lux',
       'Malphite', 'Malzahar', 'Maokai', 'MasterYi', 'MissFortune',
       'Mordekaiser', 'Morgana', 'Nami', 'Nasus', 'Nautilus', 'Neeko',
       'Nocturne', 'Nunu', 'Olaf', 'Orianna', 'Ornn', 'Pantheon', 'Poppy',
       'Pyke', 'Rakan', 'RekSai', 'Renekton', 'Rumble', 'Samira', 'Senna',
       'Seraphine', 'Sett', 'Shaco', 'Shen', 'Shyvana', 'Singed', 'Sion',
       'Sk

In [16]:
if your_pick in['',None]:
    bb = fuerzas.netforce.loc[lambda x: x['counter']==fuerzas.strongest_counter].sort_values('net_force',ascending = False)[:5]
else:
    bb = fuerzas.netforce.loc[lambda x: x['counter']==your_counter].sort_values('net_force',ascending = False)[:5]
bb

,net_force,pick,counter
"(Sett, Ezreal)",-0.041667,Sett,Ezreal
"(Pantheon, Ezreal)",-0.050000,Pantheon,Ezreal
"(Kayn, Ezreal)",-0.050000,Kayn,Ezreal
"(Volibear, Ezreal)",-0.050000,Volibear,Ezreal
"(Jax, Ezreal)",-0.050000,Jax,Ezreal


In [17]:
fuerzas.helpful.to_frame().reset_index().rename(columns = {0:'helpful'})

,championName,helpful
0,Warwick,0.083333
1,Caitlyn,0.058333
2,Jhin,0.058333
3,Yasuo,0.058333
4,Veigar,0.058333
...,...,...
94,Orianna,0.008333
95,Ornn,0.008333
96,Renekton,0.008333
97,Samira,0.008333


In [18]:
fuerzas.useless.to_frame().reset_index().rename(columns={0:'useless'})

,championName,useless
0,MissFortune,0.066667
1,Viego,0.058333
2,Tristana,0.058333
3,Smolder,0.050000
4,Nautilus,0.050000
...,...,...
97,Lulu,0.008333
98,Syndra,0.008333
99,Talon,0.008333
100,Galio,0.008333


OLIMPICOS

In [19]:
fuerzas.utility().round(10).sort_values(['utility','force_h'],ascending=[False,False])[:5]

,championName,force_h,force_u,utility
0,Warwick,0.083333,0.016667,0.066667
4,Veigar,0.058333,0.000000,0.058333
5,Jinx,0.058333,0.016667,0.041667
8,Nami,0.041667,0.000000,0.041667
2,Jhin,0.058333,0.025000,0.033333


QUESOS

In [20]:
fuerzas.utility().round(10).sort_values(['utility','force_u'],ascending=[True,False])[:5]

,championName,force_h,force_u,utility
36,MissFortune,0.016667,0.066667,-0.050000
52,Viego,0.016667,0.058333,-0.041667
68,Smolder,0.008333,0.050000,-0.041667
99,Jax,0.000000,0.041667,-0.041667
26,Tristana,0.025000,0.058333,-0.033333


In [21]:
fuerzas.force_dict

{'net_force':                      net_force  pick      counter
 (Sett, Ezreal)       -0.041667  Sett       Ezreal
 (Sett, Lux)          -0.041667  Sett          Lux
 (Sett, MissFortune)  -0.033333  Sett  MissFortune
 (Sett, Pyke)         -0.033333  Sett         Pyke
 (Sett, Yone)         -0.025000  Sett         Yone
 ...                        ...   ...          ...
 (Zed, Gnar)           0.000000   Zed         Gnar
 (Zed, Gragas)         0.000000   Zed       Gragas
 (Zed, Illaoi)         0.000000   Zed       Illaoi
 (Zed, Irelia)         0.000000   Zed       Irelia
 (Zed, Aatrox)         0.000000   Zed       Aatrox
 
 [4797 rows x 3 columns],
 'positive_force': championName
 Sett            0.033333
 Jax             0.025000
 Pantheon        0.025000
 Kayn            0.025000
 Volibear        0.025000
 Ekko            0.016667
 Senna           0.016667
 Vayne           0.016667
 Sylas           0.016667
 Warwick         0.016667
 Renekton        0.016667
 Yone            0.016667
 Na